## imports

In [23]:
import requests
import pandas as pd
import numpy as np
import mysql.connector

import sqlalchemy
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar

In [2]:
import urllib.parse as urlparse
from urllib.parse import urlencode

In [123]:
polygon_api_key = "KkfCQ7fsZnx0yK4bhX9fD81QplTh0Pf3"
_POLYGON_V2_BASE_URL = "https://api.polygon.io/v2"
_POLYGON_V3_BASE_URL = "https://api.polygon.io/v3"

In [4]:
calendar = get_calendar("NYSE")
trading_dates = calendar.schedule(
    start_date = "2023-05-01", 
    end_date = (datetime.today()-timedelta(days = 1))
).index.strftime("%Y-%m-%d").values

/var/folders/hh/x43xkncs2k7dm7t4vg3lt2cm0000gn/T/ipykernel_41849/2813920380.py:2: FutureWarning: DatetimeIndex.union_many is deprecated and will be removed in a future version. Use obj.union instead.
  trading_dates = calendar.schedule(
/var/folders/hh/x43xkncs2k7dm7t4vg3lt2cm0000gn/T/ipykernel_41849/2813920380.py:2: FutureWarning: DatetimeIndex.union_many is deprecated and will be removed in a future version. Use obj.union instead.
  trading_dates = calendar.schedule(
/var/folders/hh/x43xkncs2k7dm7t4vg3lt2cm0000gn/T/ipykernel_41849/2813920380.py:2: FutureWarning: DatetimeIndex.union_many is deprecated and will be removed in a future version. Use obj.union instead.
  trading_dates = calendar.schedule(


In [405]:
def add_params_to_url(url, params):
    url_parts = list(urlparse.urlparse(url))
    query = dict(urlparse.parse_qsl(url_parts[4]))
    query.update(params)
    url_parts[4] = urlencode(query)
    return urlparse.urlunparse(url_parts)

def add_default_ticker_params_to_polygon_url(url, additional_params = None):
    params = {
        'adjusted': 'true',
        'sort': 'asc',
        'limit': '50000',
        'apiKey': polygon_api_key,
    }
    if additional_params is not None:
        params.update(additional_params)
    return add_params_to_url(url, params)

def add_default_options_params_to_polygon_url(url, additional_params = None):
    params = {
        'limit': '1000',
        'apiKey': polygon_api_key,
    }
    if additional_params is not None:
        params.update(additional_params)
    return add_params_to_url(url, params)

def get_polygon_range_query_url(ticker, date_str_begin, date_str_end):
    return add_default_ticker_params_to_polygon_url(f"{_POLYGON_V2_BASE_URL}/aggs/ticker/{ticker}/range/1/day/{date_str_begin}/{date_str_end}")

def get_polygon_intraday_query_url(ticker, date_str):
    return add_default_ticker_params_to_polygon_url(f"{_POLYGON_V2_BASE_URL}/aggs/ticker/{ticker}/range/1/minute/{date_str}/{date_str}")

def get_polygon_options_contracts_query_url(underlying_ticker, option_type, date_str, exp_date_str):
    params = {
        "underlying_ticker": underlying_ticker,
        "contract_type": option_type,
        "as_of": date_str,
        "expiration_date": exp_date_str,        
    }
    return add_default_options_params_to_polygon_url(f"{_POLYGON_V3_BASE_URL}/reference/options/contracts", additional_params = params)

def get_polygon_result_dict(url):
    js = requests.get(url).json()
    if "results" not in js:
        return {}
    return js["results"]

def polygon_result_to_dataframe(result):
    df = pd.json_normalize(result)
    if df.empty:
        return df
    if "t" in df.columns:
        df = df.set_index("t")
        df.index = pd.to_datetime(df.index, unit="ms", utc=True).tz_convert("America/New_York")
        #df['date'] = df.index.strftime('%Y-%m-%d')
    return df

def polygon_url_to_dataframe(url):
    return polygon_result_to_dataframe(get_polygon_result_dict(url))    

In [68]:
ticker_spy = "SPY"
ticker_spx = "I:SPX"
ticker_vix1d = "I:VIX1D"
options_ticker = "SPX"

i = 2
date = trading_dates[i]
prior_day = trading_dates[i-1]

trade_list = []
times = []

## cache data

In [349]:
df_spx_daily_history = polygon_result_to_dataframe(get_polygon_result_dict(
    get_polygon_range_query_url(ticker_spx, "2020-01-01", trading_dates[-1])))
df_spx_daily_history.index = df_spx_daily_history.index.strftime("%Y-%m-%d")

In [350]:
df_spx_daily_history.to_pickle('market_data/df_spx_daily_history.pkl')

In [351]:
df_spy_daily_history = polygon_result_to_dataframe(get_polygon_result_dict(
    get_polygon_range_query_url(ticker_spy, "2020-01-01", trading_dates[-1])))
df_spy_daily_history.index = df_spy_daily_history.index.strftime("%Y-%m-%d")

In [352]:
df_spy_daily_history.to_pickle('market_data/df_spy_daily_history.pkl')

In [404]:
def get_df_intrady_history(ticker):
    dfs = []
    for date in trading_dates[1:]:
        df_intraday = polygon_result_to_dataframe(get_polygon_result_dict(
            get_polygon_intraday_query_url(ticker, date)))
        dfs.append(df_intraday.reset_index())
    df = pd.concat(dfs, ignore_index=True).set_index("t")
    return df

In [272]:
df_spx_intraday_history = get_df_intrady_history(ticker_spx)

date='2023-05-02'
date='2023-05-03'
date='2023-05-04'
date='2023-05-05'
date='2023-05-08'
date='2023-05-09'
date='2023-05-10'
date='2023-05-11'
date='2023-05-12'
date='2023-05-15'
date='2023-05-16'
date='2023-05-17'
date='2023-05-18'
date='2023-05-19'
date='2023-05-22'
date='2023-05-23'
date='2023-05-24'
date='2023-05-25'
date='2023-05-26'
date='2023-05-30'
date='2023-05-31'
date='2023-06-01'
date='2023-06-02'
date='2023-06-05'
date='2023-06-06'
date='2023-06-07'
date='2023-06-08'
date='2023-06-09'
date='2023-06-12'
date='2023-06-13'
date='2023-06-14'
date='2023-06-15'
date='2023-06-16'
date='2023-06-19'
date='2023-06-20'
date='2023-06-21'
date='2023-06-22'
date='2023-06-23'
date='2023-06-26'
date='2023-06-27'
date='2023-06-28'
date='2023-06-29'
date='2023-06-30'
date='2023-07-03'
date='2023-07-05'
date='2023-07-06'
date='2023-07-07'
date='2023-07-10'
date='2023-07-11'
date='2023-07-12'
date='2023-07-13'
date='2023-07-14'
date='2023-07-17'
date='2023-07-18'
date='2023-07-19'
date='2023

In [289]:
df_spx_intraday_history = df_spx_intraday_history.drop(columns=["index"])

In [290]:
df_spx_intraday_history.to_pickle('market_data/df_spx_intraday_history.pkl')

In [275]:
df_vix1d_intraday_history = get_df_intrady_history(ticker_vix1d)

date='2023-05-02'
date='2023-05-03'
date='2023-05-04'
date='2023-05-05'
date='2023-05-08'
date='2023-05-09'
date='2023-05-10'
date='2023-05-11'
date='2023-05-12'
date='2023-05-15'
date='2023-05-16'
date='2023-05-17'
date='2023-05-18'
date='2023-05-19'
date='2023-05-22'
date='2023-05-23'
date='2023-05-24'
date='2023-05-25'
date='2023-05-26'
date='2023-05-30'
date='2023-05-31'
date='2023-06-01'
date='2023-06-02'
date='2023-06-05'
date='2023-06-06'
date='2023-06-07'
date='2023-06-08'
date='2023-06-09'
date='2023-06-12'
date='2023-06-13'
date='2023-06-14'
date='2023-06-15'
date='2023-06-16'
date='2023-06-19'
date='2023-06-20'
date='2023-06-21'
date='2023-06-22'
date='2023-06-23'
date='2023-06-26'
date='2023-06-27'
date='2023-06-28'
date='2023-06-29'
date='2023-06-30'
date='2023-07-03'
date='2023-07-05'
date='2023-07-06'
date='2023-07-07'
date='2023-07-10'
date='2023-07-11'
date='2023-07-12'
date='2023-07-13'
date='2023-07-14'
date='2023-07-17'
date='2023-07-18'
date='2023-07-19'
date='2023

In [292]:
df_vix1d_intraday_history = df_vix1d_intraday_history.drop(columns=["index"])

In [293]:
df_vix1d_intraday_history.to_pickle('market_data/df_vix1d_intraday_history.pkl')

In [278]:
df_spy_intraday_history = get_df_intrady_history(ticker_spy)

date='2023-05-02'
date='2023-05-03'
date='2023-05-04'
date='2023-05-05'
date='2023-05-08'
date='2023-05-09'
date='2023-05-10'
date='2023-05-11'
date='2023-05-12'
date='2023-05-15'
date='2023-05-16'
date='2023-05-17'
date='2023-05-18'
date='2023-05-19'
date='2023-05-22'
date='2023-05-23'
date='2023-05-24'
date='2023-05-25'
date='2023-05-26'
date='2023-05-30'
date='2023-05-31'
date='2023-06-01'
date='2023-06-02'
date='2023-06-05'
date='2023-06-06'
date='2023-06-07'
date='2023-06-08'
date='2023-06-09'
date='2023-06-12'
date='2023-06-13'
date='2023-06-14'
date='2023-06-15'
date='2023-06-16'
date='2023-06-19'
date='2023-06-20'
date='2023-06-21'
date='2023-06-22'
date='2023-06-23'
date='2023-06-26'
date='2023-06-27'
date='2023-06-28'
date='2023-06-29'
date='2023-06-30'
date='2023-07-03'
date='2023-07-05'
date='2023-07-06'
date='2023-07-07'
date='2023-07-10'
date='2023-07-11'
date='2023-07-12'
date='2023-07-13'
date='2023-07-14'
date='2023-07-17'
date='2023-07-18'
date='2023-07-19'
date='2023

In [294]:
df_spy_intraday_history = df_spy_intraday_history.drop(columns=["index"])

In [295]:
df_spy_intraday_history.to_pickle('market_data/df_spy_intraday_history.pkl')

In [413]:
def get_df_options_history(ticker, side):
    dfs = []
    for date in trading_dates[1:]:
        df = polygon_url_to_dataframe(get_polygon_option_contracts_query_url(
            ticker, side, date, date))
        df['date'] = date
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    return df

In [414]:
df_spx_call_options_history = get_df_options_history("SPX", "call")

In [415]:
df_spx_call_options_history.to_pickle('market_data/df_spx_call_options_history.pkl')

In [416]:
df_spx_put_options_history = get_df_options_history("SPX", "put")

In [417]:
df_spx_put_options_history.to_pickle('market_data/df_spx_put_options_history.pkl')

In [602]:
df_spx_call_otm_options_history = df_spx_call_options_history.join(df_algo, on='expiration_date')
df_spx_call_otm_options_history = df_spx_call_otm_options_history[
    df_spx_call_otm_options_history.strike_price > df_spx_call_otm_options_history.upper_price
]

In [699]:
def get_df_market_open_from_intraday(df_intraday):
    df_market_open = df_intraday[df_intraday.index.time == pd.Timestamp("09:35").time()][["c"]]
    df_market_open.index = df_market_open.index.strftime("%Y-%m-%d")     
    return df_market_open

def get_df_market_close_from_intraday(df_intraday):
    df_market_close = df_intraday.tail(1)[["c"]]
    df_market_close.index = df_market_close.index.strftime("%Y-%m-%d")
    return df_market_close

In [714]:
def concat_otm_short_long(df_otm_short, df_otm_long, o_or_c, option_type):
    df_otm_options = pd.concat([
        df_otm_short.add_suffix(f"_market_{o_or_c}_s_{option_type}"), 
        df_otm_long.add_suffix(f"_market_{o_or_c}_l_{option_type}")], axis = 1)
    df_otm_options[f"market_{o_or_c}_spread"] = df_otm_options[f"c_market_{o_or_c}_s_{option_type}"] - \
        df_otm_options[f"c_market_{o_or_c}_l_{option_type}"]
    return df_otm_options

In [764]:
def get_df_spx_call_otm_options_spread(date_str):
    df_spx_call_otm_options_daily = df_spx_call_otm_options_history[df_spx_call_otm_options_history.expiration_date == date_str]
    if df_spx_call_otm_options_daily.empty:
        return None
    df_spx_call_otm_short_options_ticker = df_spx_call_otm_options_daily.head(1).ticker.values[0]
    df_spx_call_otm_long_options_ticker = df_spx_call_otm_options_daily.head(2).tail(1).ticker.values[0]
    
    df_spx_call_otm_short_option_intraday = polygon_url_to_dataframe(get_polygon_intraday_query_url(df_spx_call_otm_short_options_ticker, date_str))
    df_spx_call_otm_long_option_intraday = polygon_url_to_dataframe(get_polygon_intraday_query_url(df_spx_call_otm_long_options_ticker, date_str))
    
    if df_spx_call_otm_short_option_intraday.empty or df_spx_call_otm_long_option_intraday.empty:
        return None
    
    df_spx_call_otm_short_option_market_open = get_df_market_open_from_intraday(df_spx_call_otm_short_option_intraday)
    df_spx_call_otm_long_option_market_open = get_df_market_open_from_intraday(df_spx_call_otm_long_option_intraday)
    df_spx_call_otm_options_market_open = concat_otm_short_long(
        df_spx_call_otm_short_option_market_open, df_spx_call_otm_long_option_market_open,
        "o", "call"
    )

    df_spx_call_otm_short_option_market_close = get_df_market_close_from_intraday(df_spx_call_otm_short_option_intraday)
    df_spx_call_otm_long_option_market_close = get_df_market_close_from_intraday(df_spx_call_otm_long_option_intraday)
    df_spx_call_otm_options_market_close = concat_otm_short_long(
        df_spx_call_otm_short_option_market_close, df_spx_call_otm_long_option_market_close,
        "c", "call"
    )    
    
    df_spx_call_otm_options = df_spx_call_otm_options_market_open.join(df_spx_call_otm_options_market_close)
    df_spx_call_otm_options["pnl"] = df_spx_call_otm_options["market_o_spread"] - df_spx_call_otm_options["market_c_spread"]
    
    return df_spx_call_otm_options

In [734]:
def get_df_spx_call_otm_options_spread_history():
    dfs = []
    for date in trading_dates[1:]:
        df_date = get_df_spx_call_otm_options_spread(date)
        if df_date is None:
            continue
        dfs.append(df_date)
    df = pd.concat(dfs, ignore_index=False)
    return df

In [735]:
df_spx_otm_call_options_spread_history = get_df_spx_call_otm_options_spread_history()

In [736]:
df_spx_otm_call_options_spread_history.to_pickle('market_data/df_spx_otm_call_options_spread_history.pkl')

In [766]:
df_spx_otm_call_options_spread_history

,c_market_o_s_call,c_market_o_l_call,market_o_spread,c_market_c_s_call,c_market_c_l_call,market_c_spread,pnl
t,,,,,,,
2023-05-02,2.35,1.45,0.90,0.05,0.05,0.00,0.90
2023-05-03,7.71,6.40,1.31,0.05,0.05,0.00,1.31
2023-05-04,4.70,3.60,1.10,0.05,0.05,0.00,1.10
2023-05-05,4.00,2.85,1.15,11.00,4.00,7.00,-5.85
2023-05-08,2.65,1.82,0.83,0.05,0.05,0.00,0.83
...,...,...,...,...,...,...,...
2024-06-26,3.15,2.10,1.05,3.50,0.32,3.18,-2.13
2024-06-27,3.00,1.90,1.10,0.05,0.05,0.00,1.10
2024-06-28,4.45,3.20,1.25,0.05,0.05,0.00,1.25


In [727]:
df_spx_put_otm_options_history = df_spx_put_options_history.join(df_algo, on='expiration_date')
df_spx_put_otm_options_history = df_spx_put_otm_options_history[
    df_spx_put_otm_options_history.strike_price < df_spx_put_otm_options_history.lower_price
]

In [791]:
def get_df_spx_otm_put_options_spread(date_str):
    df_spx_put_otm_options_daily = df_spx_put_otm_options_history[df_spx_put_otm_options_history.expiration_date == date_str]
    if df_spx_put_otm_options_daily.empty:
        return None
    df_spx_put_otm_short_options_ticker = df_spx_put_otm_options_daily.tail(1).ticker.values[0]
    df_spx_put_otm_long_options_ticker = df_spx_put_otm_options_daily.tail(2).head(1).ticker.values[0]
    
    df_spx_put_otm_short_option_intraday = polygon_url_to_dataframe(get_polygon_intraday_query_url(df_spx_put_otm_short_options_ticker, date_str))
    df_spx_put_otm_long_option_intraday = polygon_url_to_dataframe(get_polygon_intraday_query_url(df_spx_put_otm_long_options_ticker, date_str))
    
    if df_spx_put_otm_short_option_intraday.empty or df_spx_put_otm_long_option_intraday.empty:
        return None
    
    df_spx_put_otm_short_option_market_open = get_df_market_open_from_intraday(df_spx_put_otm_short_option_intraday)
    df_spx_put_otm_long_option_market_open = get_df_market_open_from_intraday(df_spx_put_otm_long_option_intraday)
    df_spx_put_otm_options_market_open = concat_otm_short_long(
        df_spx_put_otm_short_option_market_open, df_spx_put_otm_long_option_market_open,
        "o", "put"
    )

    df_spx_put_otm_short_option_market_close = get_df_market_close_from_intraday(df_spx_put_otm_short_option_intraday)
    df_spx_put_otm_long_option_market_close = get_df_market_close_from_intraday(df_spx_put_otm_long_option_intraday)
    df_spx_put_otm_options_market_close = concat_otm_short_long(
        df_spx_put_otm_short_option_market_close, df_spx_put_otm_long_option_market_close,
        "c", "put"
    )    
    
    df_spx_put_otm_options = df_spx_put_otm_options_market_open.join(df_spx_put_otm_options_market_close)
    df_spx_put_otm_options["pnl"] = df_spx_put_otm_options["market_o_spread"] - df_spx_put_otm_options["market_c_spread"]
    
    return df_spx_put_otm_options

In [792]:
def get_df_spx_otm_put_options_spread_history():
    dfs = []
    for date in trading_dates[1:]:
        df_date = get_df_spx_otm_put_options_spread(date)
        if df_date is None:
            continue
        dfs.append(df_date)
    df = pd.concat(dfs, ignore_index=False)
    return df

In [747]:
df_spx_otm_put_options_spread_history = get_df_spx_put_otm_options_spread_history()


KeyboardInterrupt



In [748]:
df_spx_otm_put_options_spread_history.to_pickle('market_data/df_spx_otm_put_options_spread_history.pkl')

In [795]:
df_spx_otm_put_options_spread_history

,c_market_o_s_put,c_market_o_l_put,market_o_spread,c_market_c_s_put,c_market_c_l_put,market_c_spread,pnl
t,,,,,,,
2023-05-02,3.90,2.80,1.10,20.20,16.94,3.26,-2.16
2023-05-03,9.29,7.91,1.38,8.00,3.00,5.00,-3.62
2023-05-04,6.50,5.32,1.18,0.15,0.05,0.10,1.08
2023-05-05,5.80,4.70,1.10,0.05,0.05,0.00,1.10
2023-05-08,3.65,2.80,0.85,0.05,0.05,0.00,0.85
...,...,...,...,...,...,...,...
2024-06-26,3.35,2.45,0.90,0.05,0.05,0.00,0.90
2024-06-27,3.49,2.65,0.84,0.05,0.05,0.00,0.84
2024-06-28,5.50,4.31,1.19,23.25,18.30,4.95,-3.76


## debug

In [619]:
df_spx_put_otm_options_daily = df_spx_put_otm_options_history[df_spx_put_otm_options_history.expiration_date == "2024-06-11"]

In [621]:
get_df_spx_put_otm_options_spread("2024-06-10")

,c_short_put,c_long_put,spread_value
date,,,
2024-06-11,3.6,2.75,0.85


In [631]:
polygon_url_to_dataframe(get_polygon_intraday_query_url("O:SPXW240610C05350000", "2024-06-10"))

,v,vw,o,c,h,l,n
t,,,,,,,
2024-06-10 09:30:00-04:00,101,4.7689,4.79,4.70,5.00,4.58,60
2024-06-10 09:31:00-04:00,289,4.1428,4.70,3.97,4.80,3.70,116
2024-06-10 09:32:00-04:00,534,3.8347,3.90,3.90,4.00,3.61,206
2024-06-10 09:33:00-04:00,898,3.5538,3.85,3.50,3.85,3.22,147
2024-06-10 09:34:00-04:00,336,3.6096,3.50,3.80,3.82,3.35,81
...,...,...,...,...,...,...,...
2024-06-10 15:27:00-04:00,17,11.2147,11.07,11.40,11.58,10.80,10
2024-06-10 15:28:00-04:00,19,11.8463,11.20,12.20,12.20,11.20,10
2024-06-10 15:29:00-04:00,117,12.6928,11.90,12.90,12.90,11.70,11


In [634]:
polygon_url_to_dataframe(get_polygon_intraday_query_url("O:SPXW240610C05355000", "2024-06-10"))

,v,vw,o,c,h,l,n
t,,,,,,,
2024-06-10 09:30:00-04:00,84,3.3186,3.40,3.20,3.45,3.20,47
2024-06-10 09:31:00-04:00,191,2.8497,3.10,2.70,3.33,2.53,93
2024-06-10 09:32:00-04:00,217,2.5967,2.63,2.65,2.70,2.45,87
2024-06-10 09:33:00-04:00,311,2.3432,2.55,2.27,2.55,2.19,108
2024-06-10 09:34:00-04:00,159,2.3766,2.30,2.50,2.55,2.20,63
...,...,...,...,...,...,...,...
2024-06-10 15:27:00-04:00,36,6.7631,6.90,6.85,7.27,6.60,24
2024-06-10 15:28:00-04:00,32,7.2866,6.70,7.80,7.90,6.64,17
2024-06-10 15:29:00-04:00,40,7.8862,7.80,8.10,8.30,7.50,21


In [635]:
date = "2024-06-10"

In [638]:
df_spx_call_otm_options_history = df_spx_call_options_history.join(df_algo, on='expiration_date')

In [639]:
df_spx_call_otm_options_history = df_spx_call_otm_options_history[
    df_spx_call_otm_options_history.strike_price > df_spx_call_otm_options_history.upper_price
]

In [647]:
df_spx_call_otm_options_history[df_spx_call_otm_options_history.expiration_date == date].head(2)

,cfi,contract_type,exercise_style,expiration_date,primary_exchange,shares_per_contract,strike_price,ticker,underlying_ticker,date,c_daily,c_market_open,regime,expected_move,lower_price,upper_price
58676,OCEICS,call,european,2024-06-10,XCBO,100.0,5350.0,O:SPXW240610C05350000,SPX,2024-06-10,5360.79,5335.97,1,0.0024,5323.16,5348.78
58677,OCEICS,call,european,2024-06-10,XCBO,100.0,5355.0,O:SPXW240610C05355000,SPX,2024-06-10,5360.79,5335.97,1,0.0024,5323.16,5348.78


In [648]:
df_spx_call_otm_short_options_ticker = df_spx_call_otm_options_history[df_spx_call_otm_options_history.expiration_date == date].head(1).ticker.values[0]
df_spx_call_otm_long_options_ticker = df_spx_call_otm_options_history[df_spx_call_otm_options_history.expiration_date == date].head(2).tail(1).ticker.values[0]

In [649]:
df_spx_call_otm_short_option_intraday = polygon_url_to_dataframe(get_polygon_intraday_query_url(df_spx_call_otm_short_options_ticker, date))

In [650]:
df_spx_call_otm_long_option_intraday = polygon_url_to_dataframe(get_polygon_intraday_query_url(df_spx_call_otm_long_options_ticker, date))

In [660]:
df_spx_call_otm_short_option_market_close = df_spx_call_otm_short_option_intraday.tail(1)[["c"]]
df_spx_call_otm_short_option_market_close.index = df_spx_call_otm_short_option_market_close.index.strftime("%Y-%m-%d")

In [661]:
df_spx_call_otm_short_option_market_close

,c
t,
2024-06-10,9.15


In [651]:
df_spx_call_otm_short_option_market_open = df_spx_call_otm_short_option_intraday[df_spx_call_otm_short_option_intraday.index.time == pd.Timestamp("09:35").time()][["c"]]
df_spx_call_otm_short_option_market_open.index = df_spx_call_otm_short_option_market_open.index.strftime("%Y-%m-%d")

In [652]:
df_spx_call_otm_short_option_market_open

,c
t,
2024-06-10,3.8


In [534]:
df_spx_call_otm_long_option_market_open = df_spx_call_otm_long_option_intraday[df_spx_call_otm_long_option_intraday.index.time == pd.Timestamp("09:35").time()][["c"]]
df_spx_call_otm_long_option_market_open.index = df_spx_call_otm_long_option_market_open.index.strftime("%Y-%m-%d")

In [535]:
df_spx_call_otm_long_option_market_open

,c
t,
2024-06-10,2.55


In [541]:
df_spx_call_otm_options_market_open = pd.concat([df_spx_call_otm_short_option_market_open.add_suffix("_short_call"), df_spx_call_otm_long_option_market_open.add_suffix("_long_call")], axis = 1)

In [543]:
df_spx_call_otm_options_market_open["spread_value"] = df_spx_call_otm_options_market_open["c_short_call"] - df_spx_call_otm_options_market_open["c_long_call"]

In [544]:
df_spx_call_otm_options_market_open

,c_short_call,c_long_call,spread_value
t,,,
2024-06-10,3.8,2.55,1.25


In [526]:
df_algo[df_algo.index == '2024-06-10']

,c_daily,c_market_open,regime,expected_move,lower_price,upper_price
t,,,,,,
2024-06-10,5360.79,5335.97,1,0.0024,5323.16,5348.78


## read cached data

In [315]:
df_spx_daily_history = pd.read_pickle('market_data/df_spx_daily_history.pkl')
df_spy_daily_history = pd.read_pickle('market_data/df_spy_daily_history.pkl')
df_spx_intraday_history = pd.read_pickle('market_data/df_spx_intraday_history.pkl')
df_vix1d_intraday_history = pd.read_pickle('market_data/df_vix1d_intraday_history.pkl')
df_spy_intraday_history = pd.read_pickle('market_data/df_spy_intraday_history.pkl')

In [ ]:
df_spx_call_options_history = pd.read_pickle('market_data/df_spx_call_options_history.pkl')
df_spx_put_options_history = pd.read_pickle('market_data/df_spx_put_options_history.pkl')

In [ ]:
df_spx_otm_call_options_spread_history = pd.read_pickle('market_data/df_spx_otm_call_options_spread_history.pkl')
df_spx_otm_put_options_spread_history = pd.read_pickle('market_data/df_spx_otm_put_options_spread_history.pkl')

## logic

In [388]:
df_spx_market_open_history = df_spx_intraday_history[df_spx_intraday_history.index.time == pd.Timestamp("09:35").time()][["c"]]
df_spx_market_open_history.index = df_spx_market_open_history.index.strftime("%Y-%m-%d")

df_vix1d_market_open_history = df_vix1d_intraday_history[df_vix1d_intraday_history.index.time == pd.Timestamp("09:35").time()][["c"]]
df_vix1d_market_open_history.index = df_vix1d_market_open_history.index.strftime("%Y-%m-%d")

df_spy_market_open_history = df_spy_intraday_history[df_spy_intraday_history.index.time == pd.Timestamp("09:35").time()][["c"]]
df_spy_market_open_history.index = df_spy_market_open_history.index.strftime("%Y-%m-%d")

In [471]:
df_algo = df_spx_daily_history[["c"]].copy()

In [472]:
df_algo = df_algo.join(df_spx_market_open_history, lsuffix="_daily", rsuffix="_market_open")

In [754]:
df_spy = df_spy_daily_history[["c"]].copy()

In [755]:
df_spy["1_mo_avg"] = df_spy["c"].rolling(window=20).mean()
df_spy["3_mo_avg"] = df_spy["c"].rolling(window=60).mean()
df_spy['regime'] = df_spy.apply(lambda row: 1 if (row['c'] > row['1_mo_avg']) else -1, axis=1)

In [756]:
df_algo['regime'] = df_spy['regime']

In [761]:
df_algo["expected_move"] = (round(df_vix1d_market_open_history / np.sqrt(252), 2) / 100) * 0.5
df_algo["lower_price"] = round(df_algo['c_market_open'] * (1.0 - df_algo['expected_move']), 2)
df_algo["upper_price"] = round(df_algo['c_market_open'] * (1.0 + df_algo['expected_move']), 2)

In [798]:
df_algo = df_algo.join(df_spx_otm_call_options_spread_history[["pnl"]].add_suffix("_call")).join(
    df_spx_put_otm_options_spread_history[["pnl"]].add_suffix("_put")
)

In [799]:
df_algo

,c_daily,c_market_open,regime,expected_move,lower_price,upper_price,pnl_call,pnl_put
t,,,,,,,,
2023-02-16,4090.41,NaN,1,NaN,NaN,NaN,NaN,NaN
2023-03-06,4048.42,NaN,-1,NaN,NaN,NaN,NaN,NaN
2023-03-07,3986.37,NaN,-1,NaN,NaN,NaN,NaN,NaN
2023-03-08,3992.01,NaN,-1,NaN,NaN,NaN,NaN,NaN
2023-03-09,3918.32,NaN,-1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-06-26,5477.90,5457.73,1,0.00235,5444.90,5470.56,-2.13,0.90
2024-06-27,5482.87,5483.08,1,0.00240,5469.92,5496.24,1.10,0.84
2024-06-28,5460.48,5496.61,1,0.00280,5481.22,5512.00,1.25,-3.76


In [800]:
df_algo.groupby("regime")[["pnl_call", "pnl_put"]].sum()

,pnl_call,pnl_put
regime,,
-1,33.14,-74.90
1,-54.36,100.26


In [782]:
df_spx_otm_call_options_spread_history[
    df_algo["regime"] < 0
].sum()

/var/folders/hh/x43xkncs2k7dm7t4vg3lt2cm0000gn/T/ipykernel_41849/2928183103.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_spx_otm_call_options_spread_history[


c_market_o_s_call    323.67
c_market_o_l_call    239.52
market_o_spread       84.15
c_market_c_s_call    196.58
c_market_c_l_call    145.57
market_c_spread       51.01
pnl                   33.14
dtype: float64

In [783]:
df_spx_otm_call_options_spread_history.sum()

c_market_o_s_call    1047.16
c_market_o_l_call     732.10
market_o_spread       315.06
c_market_c_s_call    1132.63
c_market_c_l_call     796.35
market_c_spread       336.28
pnl                   -21.22
dtype: float64

In [781]:
df_spx_put_otm_options_spread_history[
    df_algo["regime"] > 0
].sum()

/var/folders/hh/x43xkncs2k7dm7t4vg3lt2cm0000gn/T/ipykernel_41849/2924966555.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_spx_put_otm_options_spread_history[


c_market_o_s_put    896.91
c_market_o_l_put    668.93
market_o_spread     227.98
c_market_c_s_put    478.88
c_market_c_l_put    351.16
market_c_spread     127.72
pnl                 100.26
dtype: float64

In [784]:
df_spx_put_otm_options_spread_history.sum()

c_market_o_s_put    1306.32
c_market_o_l_put     997.03
market_o_spread      309.29
c_market_c_s_put    1205.80
c_market_c_l_put     921.87
market_c_spread      283.93
pnl                   25.36
dtype: float64

## original codes

In [ ]:
underlying_data = underlying_data[underlying_data.index.time >= pd.Timestamp("09:35").time()].copy()
index_data = index_data[index_data.index.time >= pd.Timestamp("09:35").time()].copy()
etf_underlying_data = etf_underlying_data[etf_underlying_data.index.time >= pd.Timestamp("09:35").time()].copy()

index_price = index_data["c"].iloc[0]        
price = underlying_data["c"].iloc[0]

expected_move = (round((index_price / np.sqrt(252)), 2)/100)*.50

lower_price = round(price - (price * expected_move))
upper_price = round(price + (price * expected_move))

exp_date = date

# Pull the data at 9:35 to represent the most up-to-date regime that would be available
concatenated_regime_dataset = pd.concat([big_underlying_data, etf_underlying_data.head(1)], axis = 0)
concatenated_regime_dataset["1_mo_avg"] = concatenated_regime_dataset["c"].rolling(window=20).mean()
concatenated_regime_dataset["3_mo_avg"] = concatenated_regime_dataset["c"].rolling(window=60).mean()
concatenated_regime_dataset['regime'] = concatenated_regime_dataset.apply(lambda row: 1 if (row['c'] > row['1_mo_avg']) else 0, axis=1)

direction = concatenated_regime_dataset["regime"].iloc[-1]

In [ ]:
start_time = datetime.now()

prior_day = trading_dates[np.where(trading_dates==date)[0][0]-1]

prior_day_underlying_data = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{prior_day}/{prior_day}?adjusted=true&sort=asc&limit=50000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
prior_day_underlying_data.index = pd.to_datetime(prior_day_underlying_data.index, unit="ms", utc=True).tz_convert("America/New_York")

big_underlying_data = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/SPY/range/1/day/2020-01-01/{prior_day}?adjusted=true&sort=asc&limit=50000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
big_underlying_data.index = pd.to_datetime(big_underlying_data.index, unit="ms", utc=True).tz_convert("America/New_York")
big_underlying_data["1_mo_avg"] = big_underlying_data["c"].rolling(window=20).mean()
big_underlying_data["3_mo_avg"] = big_underlying_data["c"].rolling(window=60).mean()
big_underlying_data['regime'] = big_underlying_data.apply(lambda row: 1 if (row['c'] > row['1_mo_avg']) else 0, axis=1)

underlying_data = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
underlying_data.index = pd.to_datetime(underlying_data.index, unit="ms", utc=True).tz_convert("America/New_York")

index_data = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/{index_ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
index_data.index = pd.to_datetime(index_data.index, unit="ms", utc=True).tz_convert("America/New_York")

etf_underlying_data = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/SPY/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
etf_underlying_data.index = pd.to_datetime(etf_underlying_data.index, unit="ms", utc=True).tz_convert("America/New_York")

In [ ]:
underlying_data = underlying_data[underlying_data.index.time >= pd.Timestamp("09:35").time()].copy()
index_data = index_data[index_data.index.time >= pd.Timestamp("09:35").time()].copy()
etf_underlying_data = etf_underlying_data[etf_underlying_data.index.time >= pd.Timestamp("09:35").time()].copy()

index_price = index_data["c"].iloc[0]        
price = underlying_data["c"].iloc[0]

expected_move = (round((index_price / np.sqrt(252)), 2)/100)*.50

lower_price = round(price - (price * expected_move))
upper_price = round(price + (price * expected_move))

exp_date = date

# Pull the data at 9:35 to represent the most up-to-date regime that would be available
concatenated_regime_dataset = pd.concat([big_underlying_data, etf_underlying_data.head(1)], axis = 0)
concatenated_regime_dataset["1_mo_avg"] = concatenated_regime_dataset["c"].rolling(window=20).mean()
concatenated_regime_dataset["3_mo_avg"] = concatenated_regime_dataset["c"].rolling(window=60).mean()
concatenated_regime_dataset['regime'] = concatenated_regime_dataset.apply(lambda row: 1 if (row['c'] > row['1_mo_avg']) else 0, axis=1)

direction = concatenated_regime_dataset["regime"].iloc[-1]

In [ ]:
if direction == 0:

    valid_calls = pd.json_normalize(requests.get(f"https://api.polygon.io/v3/reference/options/contracts?underlying_ticker={options_ticker}&contract_type=call&as_of={date}&expiration_date={exp_date}&limit=1000&apiKey={polygon_api_key}").json()["results"])
    valid_calls = valid_calls[valid_calls["ticker"].str.contains("SPXW")].copy()
    valid_calls["days_to_exp"] = (pd.to_datetime(valid_calls["expiration_date"]) - pd.to_datetime(date)).dt.days
    valid_calls["distance_from_price"] = abs(valid_calls["strike_price"] - price)

    otm_calls = valid_calls[valid_calls["strike_price"] >= upper_price]

    short_call = otm_calls.iloc[[0]]
    long_call = otm_calls.iloc[[1]]

    short_call_ohlcv = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/{short_call['ticker'].iloc[0]}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=1000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
    short_call_ohlcv.index = pd.to_datetime(short_call_ohlcv.index, unit = "ms", utc = True).tz_convert("America/New_York") 

    long_call_ohlcv = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/{long_call['ticker'].iloc[0]}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=1000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
    long_call_ohlcv.index = pd.to_datetime(long_call_ohlcv.index, unit = "ms", utc = True).tz_convert("America/New_York") 

    spread = pd.concat([short_call_ohlcv.add_prefix("short_call_"), long_call_ohlcv.add_prefix("long_call_")], axis = 1).dropna()
    spread = spread[spread.index.time >= pd.Timestamp("09:35").time()].copy()
    spread["spread_value"] = spread["short_call_c"] - spread["long_call_c"]
    cost = spread["spread_value"].iloc[0]

    underlying_data["distance_from_short_strike"] = round(((short_call["strike_price"].iloc[0] - underlying_data["c"]) / underlying_data["c"].iloc[0])*100, 2)


elif direction == 1:

    valid_puts = pd.json_normalize(requests.get(f"https://api.polygon.io/v3/reference/options/contracts?underlying_ticker={options_ticker}&contract_type=put&as_of={date}&expiration_date={exp_date}&limit=1000&apiKey={polygon_api_key}").json()["results"])
    valid_puts = valid_puts[valid_puts["ticker"].str.contains("SPXW")].copy()
    valid_puts["days_to_exp"] = (pd.to_datetime(valid_puts["expiration_date"]) - pd.to_datetime(date)).dt.days
    valid_puts["distance_from_price"] = abs(price - valid_puts["strike_price"])

    otm_puts = valid_puts[valid_puts["strike_price"] <= lower_price].sort_values("distance_from_price", ascending = True)

    short_put = otm_puts.iloc[[0]]
    long_put = otm_puts.iloc[[1]]

    short_put_ohlcv = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/{short_put['ticker'].iloc[0]}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=1000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
    short_put_ohlcv.index = pd.to_datetime(short_put_ohlcv.index, unit = "ms", utc = True).tz_convert("America/New_York")   

    long_put_ohlcv = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/ticker/{long_put['ticker'].iloc[0]}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=1000&apiKey={polygon_api_key}").json()["results"]).set_index("t")
    long_put_ohlcv.index = pd.to_datetime(long_put_ohlcv.index, unit = "ms", utc = True).tz_convert("America/New_York")

    spread = pd.concat([short_put_ohlcv.add_prefix("short_put_"), long_put_ohlcv.add_prefix("long_put_")], axis = 1).dropna()
    spread = spread[spread.index.time >= pd.Timestamp("09:35").time()].copy()
    spread["spread_value"] = spread["short_put_c"] - spread["long_put_c"]
    cost = spread["spread_value"].iloc[0]

    underlying_data["distance_from_short_strike"] = round(((underlying_data["c"] - short_put["strike_price"].iloc[0]) / short_put["strike_price"].iloc[0])*100, 2)

final_value = spread["spread_value"].iloc[-1]
gross_pnl = cost - final_value
gross_pnl_percent = round((gross_pnl / cost)*100,2)

trade_data = pd.DataFrame([{"date": date, "cost": cost, "final_price": final_value, "gross_pnl": gross_pnl, "gross_pnl_percent": gross_pnl_percent, "ticker": ticker, "direction": direction}])

trade_list.append(trade_data)

end_time = datetime.now()
seconds_to_complete = (end_time - start_time).total_seconds()
times.append(seconds_to_complete)
iteration = round((np.where(trading_dates==date)[0][0]/len(trading_dates))*100,2)
iterations_remaining = len(trading_dates) - np.where(trading_dates==date)[0][0]
average_time_to_complete = np.mean(times)
estimated_completion_time = (datetime.now() + timedelta(seconds = int(average_time_to_complete*iterations_remaining)))
time_remaining = estimated_completion_time - datetime.now()
print(f"{iteration}% complete, {time_remaining} left, ETA: {estimated_completion_time}")

except Exception as data_error:
print(data_error)
continue

all_trades = pd.concat(trade_list).drop_duplicates("date").set_index("date")
all_trades.index = pd.to_datetime(all_trades.index).tz_localize("America/New_York")

all_trades["max_loss"] = (5 - all_trades["cost"])
all_trades['gross_pnl'] = all_trades.apply(lambda row: row['max_loss']*-1 if (row['gross_pnl'] < row['max_loss']*-1) else row["gross_pnl"], axis=1)

all_trades["contracts"] = (5 / all_trades["max_loss"]).astype(int)
all_trades["max_loss"] = (all_trades["max_loss"]) * all_trades["contracts"]
all_trades["fees"] = all_trades["contracts"] * .04
all_trades["net_pnl"] = (all_trades["gross_pnl"] * all_trades["contracts"]) - all_trades["fees"]

all_trades["net_capital"] = 2000 + (all_trades["net_pnl"]*100).cumsum()
